# Aggreagations in Spark:
- All agg in the spark are used using built in spark functions
- As a rule of thumb remeber to use col() when we are doing any transformation on the col like year, concat, date, arithmetic operation etc but dont use col() when refering it to any function as string like for group by, window, partition by order by etc
- Simple Agg:
    - Simple agg is summarize the complete dataset
    - returns the simgle row of data
    - example cpunt, sum, avg, min, max 
    - general syntax format df.agg(
        agg1,
        agg2,
        agg3
    )
- Grouping Agg
    - this is synonym to the sql group by aggregates 
    - general syntax df.groupBy(
        [col1,
        col2]
    ).agg(
        agg1,
        agg2
    )
- Windowing Agg
    - Window aggregation means applying an aggregation function (like sum, avg, count, dense_rank, etc.) across a group of rows that are related to the current row, without collapsing them into a single row (unlike groupBy).
    - It keeps the row-level granularity but calculates aggregated values across a defined "window" (partition).

In [1]:
from pyspark.sql import SparkSession
from pyspark.sql.functions import *
from pyspark.sql.types import *

if __name__ == '__main__':

    spark = (
        SparkSession.builder
        .appName('Spark aggregations')
        .master('local[*]')
        .getOrCreate()
    )

    spark_df = (
        spark
            .read
            .format('csv')
            .option('header','true')
            .option('inferSchema','true')
            .load(
                path = r'C:\Users\shubh\OneDrive\Documents\Visual Studio 2017\datasets\tips.csv',
                encoder = 'utf-8'
            )
    )

    spark_df.show(5)
    spark_df.printSchema()

+----------+----+------+------+---+------+----+
|total_bill| tip|   sex|smoker|day|  time|size|
+----------+----+------+------+---+------+----+
|     16.99|1.01|Female|    No|Sun|Dinner|   2|
|     10.34|1.66|  Male|    No|Sun|Dinner|   3|
|     21.01| 3.5|  Male|    No|Sun|Dinner|   3|
|     23.68|3.31|  Male|    No|Sun|Dinner|   2|
|     24.59|3.61|Female|    No|Sun|Dinner|   4|
+----------+----+------+------+---+------+----+
only showing top 5 rows

root
 |-- total_bill: double (nullable = true)
 |-- tip: double (nullable = true)
 |-- sex: string (nullable = true)
 |-- smoker: string (nullable = true)
 |-- day: string (nullable = true)
 |-- time: string (nullable = true)
 |-- size: integer (nullable = true)



In [8]:
# Simple Aggregation: 

spark_df.select(
    count('*').alias('recordCount'),
    round(sum(col('total_bill')),0).alias('totalBills'),
    round(sum(col('tip')),0).alias('totalTips'),
    round(avg(col('tip')),0).alias('avgTips')
).show()

+-----------+----------+---------+-------+
|recordCount|totalBills|totalTips|avgTips|
+-----------+----------+---------+-------+
|        244|    4828.0|    732.0|    3.0|
+-----------+----------+---------+-------+



In [9]:
spark_df.withColumn(
    'recordCount',count(col('total_bill'))
).withColumn(
    'totalBillAmount',sum(col('total_bill'))
).withColumn(
    'totalTips',sum(col('tip'))
).show()

# we cannot do agg using withColumn because it returns a new column in the data frame with transformations but since aggregation are scaler this doens make sense

# You can use withColumn() for:
# Row-level transformations, i.e., logic applied to each row:

# when/otherwise (like SQL CASE WHEN)

# to_date, date_add, datediff, etc.

# concat, substring, regexp_replace, etc.

# udf-based transformations

# Arithmetic or logic using col() expressions

AnalysisException: [MISSING_GROUP_BY] The query does not include a GROUP BY clause. Add GROUP BY or turn it into the window functions using OVER clauses.;
Aggregate [total_bill#17, tip#18, sex#19, smoker#20, day#21, time#22, size#23, count(total_bill#17) AS recordCount#281L]
+- Relation [total_bill#17,tip#18,sex#19,smoker#20,day#21,time#22,size#23] csv


In [61]:
spark_df.agg(
   count(col('total_bill')).alias('recordCount'),
    sum(col('total_bill')).alias('totalBillAmount'),
    sum(col('tip')).alias('totalTips')
).show()


+-----------+-----------------+---------+
|recordCount|  totalBillAmount|totalTips|
+-----------+-----------------+---------+
|        244|4827.770000000001|   731.58|
+-----------+-----------------+---------+



In [16]:

# Grouped Aggregation:
# Returns better results as per slicig and dicing requirements

# use case lets find avg tips and avg bill by time and sex
# after group by we cannot use select as its a dataframe method and after grouping it becomes grouped dataframe object


# spark_df.groupBy(['time','sex']).select(
#     'time',
#     'sex',
#     round(avg(col('tip'))).alias('averageTip'),
#     round(avg(col('total_bill'))).alias('averageBill')
# ).show()

spark_df.groupBy(['time', 'sex']).agg(
    round(avg(col('tip')), 2).alias('averageTip'),
    round(avg(col('total_bill')), 2).alias('averageBill'),
    round(((col('averageTip')/col('averageBill'))*100),2).alias('tipPercent')
).orderBy(col('averageBill').desc()).show()

+------+------+----------+-----------+----------+
|  time|   sex|averageTip|averageBill|tipPercent|
+------+------+----------+-----------+----------+
|Dinner|  Male|      3.14|      21.46|     14.63|
|Dinner|Female|       3.0|      19.21|     15.62|
| Lunch|  Male|      2.88|      18.05|     15.96|
| Lunch|Female|      2.58|      16.34|     15.79|
+------+------+----------+-----------+----------+



In [31]:

schema = StructType([
    StructField ('Region',StringType()),
    StructField ('Country',StringType()),
    StructField ('Item Type',StringType()),
    StructField ('Sales Channel',StringType()),
    StructField ('Order Priority',StringType()),
    StructField ('Order Date',StringType()),
    StructField ('Order ID',IntegerType()),
    StructField ('Ship Date',StringType()),
    StructField ('Units Sold',IntegerType()),
    StructField ('Unit Price',FloatType()),
    StructField ('Unit Cost',FloatType()),
    StructField ('Total Revenue',DoubleType()),
    StructField ('Total Cost',FloatType()),
    StructField ('Total Profit',FloatType())
])

sales_df = (
    spark.read
        .format('csv')
        .option('inferSchema','true')
        .option('header','true')
        .load(
            path = r'C:\Users\shubh\OneDrive\Documents\Visual Studio 2017\datasets\5M Sale Transaction.csv',
            encoder = 'utf-8',
            schema = schema
        )
)

spark.sql("SET spark.sql.legacy.timeParserPolicy = LEGACY")
# gotta transform the data from string to date using function
sales_df_transformed = sales_df.withColumn(
    'Order Date',to_date('Order Date','MM/dd/yyyy')
).withColumn(
    'Ship Date',to_date('Ship Date','MM/dd/yyyy')
)

sales_df_transformed.show(5)
sales_df_transformed.printSchema()

+--------------------+-------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|  Palau|Office Supplies|       Online|             H|2016-03-06|517073523|2016-03-26|      2401|    651.21|   524.96|   1563555.21| 1260429.0|   303126.25|
|              Europe| Poland|      Beverages|       Online|             L|2010-04-18|380507028|2010-05-26|      9340|     47.45|    31.79|     443183.0|  296918.6|    146264.4|
|       North America| Canada|         Cereal|       Online|             M|2015-01-08|504055583|2015-01-31|   

In [32]:
# exaple to create a new field and then perform agg with grouping on that field
sales_df_transformed.withColumn(
    'weekOfOrder',weekofyear(col('Order Date'))
).groupBy(
    [col('Country'),col('weekOfOrder')]
).agg(
    count(col('Order ID')).alias('numOrders'),
    sum(col('Units Sold')).alias('totalQuantity'),
    round(sum(col('Total revenue')),2).alias('totalRevenue')
).orderBy(col('totalRevenue').desc()).show(5)

+----------+-----------+---------+-------------+--------------+
|   Country|weekOfOrder|numOrders|totalQuantity|  totalRevenue|
+----------+-----------+---------+-------------+--------------+
|Montenegro|         20|      545|      2895253|9.8321366431E8|
|     Gabon|         16|      558|      2849656|9.5522715972E8|
|  Malaysia|         23|      536|      2775604|9.5298383403E8|
|   Ireland|         24|      537|      2718680|9.5014640829E8|
|   Ireland|         32|      536|      2908891|9.4802659849E8|
+----------+-----------+---------+-------------+--------------+
only showing top 5 rows



In [ ]:
# Windowed Aggregations
# example to find week on week running total using sql first
spark.sql("DROP VIEW IF EXISTS global_temp.sales_view")
sales_df_transformed.createGlobalTempView('sales_view')



In [49]:
# trying first with spark sql 
spark.sql(
    """
    WITH cte as (
    SELECT
        Country,
        weekofyear(CAST(`Order Date` AS DATE)) AS week,
        COUNT(`Order ID`) AS numInvoices,
        SUM(`Units Sold`) AS totalQuantity,
        SUM(`Total Revenue`) AS totalValue    
    FROM 
        sales_view
    WHERE 
        Year(`Order Date`) = 2010
        AND MONth(`Order Date`) = 03
    GROUP BY
        Country,
        weekofyear(CAST(`Order Date` AS DATE))
    Order By Country, week)
    Select
        *,
        sum(totalValue) over (
            partition by Country order by week asc
        ) as runningTotal
    From cte
    """
).show()
# we have to use `` as '' this is considered string and function fails also if there are no space we can use without any quotation


+-----------+----+-----------+-------------+--------------------+--------------------+
|    Country|week|numInvoices|totalQuantity|          totalValue|        runningTotal|
+-----------+----+-----------+-------------+--------------------+--------------------+
|Afghanistan|   9|         44|       229879|       8.764520492E7|       8.764520492E7|
|Afghanistan|  10|         52|       270470| 7.351244774000001E7|1.6115765266000003E8|
|Afghanistan|  11|         50|       288696| 8.902117754000002E7|2.5017883020000005E8|
|Afghanistan|  12|         52|       288001|       5.803784811E7|3.0821667831000006E8|
|Afghanistan|  13|         18|       107396|3.1638664630000003E7|3.3985534294000006E8|
|    Albania|   9|         41|       214742|6.1229220059999995E7|6.1229220059999995E7|
|    Albania|  10|         42|       270596| 5.151935969999999E7|1.1274857975999999E8|
|    Albania|  11|         53|       269077| 6.478094346000001E7|      1.7752952322E8|
|    Albania|  12|         47|       249395

In [65]:
from pyspark.sql.window import Window

sales_df_transformed.filter(
    (year('Order Date') == 2010) & (month('Order Date') == 3)
).withColumn(
    'weekOfOrder',weekofyear(col('Order Date'))
).groupBy(
    ['Country','weekOfOrder']
).agg(
    count(col('Order ID')).alias('numOrders'),
    sum(col('Units Sold')).alias('totalQuantity'),
    round(sum(col('Total Revenue')),2).alias('totalRevenue')
).orderBy(
    col('totalRevenue').desc()
).withColumn(
    'runningTotal',
    sum(col('totalRevenue')).over(
        Window.partitionBy(
            'Country'
        ).orderBy(
            'weekOfOrder'
        )
    )
).show()

+-----------+-----------+---------+-------------+--------------+--------------+
|    Country|weekOfOrder|numOrders|totalQuantity|  totalRevenue|  runningTotal|
+-----------+-----------+---------+-------------+--------------+--------------+
|Afghanistan|          9|       44|       229879| 8.764520492E7| 8.764520492E7|
|Afghanistan|         10|       52|       270470| 7.351244774E7|1.6115765266E8|
|Afghanistan|         11|       50|       288696| 8.902117754E7| 2.501788302E8|
|Afghanistan|         12|       52|       288001| 5.803784811E7|3.0821667831E8|
|Afghanistan|         13|       18|       107396| 3.163866463E7|3.3985534294E8|
|    Albania|          9|       41|       214742| 6.122922006E7| 6.122922006E7|
|    Albania|         10|       42|       270596|  5.15193597E7|1.1274857976E8|
|    Albania|         11|       53|       269077| 6.478094346E7|1.7752952322E8|
|    Albania|         12|       47|       249395| 5.444069947E7|2.3197022269E8|
|    Albania|         13|       21|     

In [ ]:
# another example to find top 5 country by region for total_revenue and if same rank by profit margin for the year 2016 march month

In [67]:
sales_df_transformed.show(5)

+--------------------+-------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|              Region|Country|      Item Type|Sales Channel|Order Priority|Order Date| Order ID| Ship Date|Units Sold|Unit Price|Unit Cost|Total Revenue|Total Cost|Total Profit|
+--------------------+-------+---------------+-------------+--------------+----------+---------+----------+----------+----------+---------+-------------+----------+------------+
|Australia and Oce...|  Palau|Office Supplies|       Online|             H|2016-03-06|517073523|2016-03-26|      2401|    651.21|   524.96|   1563555.21| 1260429.0|   303126.25|
|              Europe| Poland|      Beverages|       Online|             L|2010-04-18|380507028|2010-05-26|      9340|     47.45|    31.79|     443183.0|  296918.6|    146264.4|
|       North America| Canada|         Cereal|       Online|             M|2015-01-08|504055583|2015-01-31|   

In [70]:
sales_df_transformed.filter(
    (year(col('Order Date')) == 2016) & (month(col('Order Date')) == 3)
).withColumn(
    'profitMargin',
    (col('Total Profit')/col('Total Revenue'))
).groupBy(
    [
        'Region',
        'Country'
    ]
).agg(
    sum(col('Total Revenue')).alias('totalRevenue'),
    sum(col('profitMargin')).alias('totalProfitMargin')
).withColumn(
    'Rank',
    dense_rank().over(
        Window.partitionBy(
            'Region'
        ).orderBy(
           [
               col('totalRevenue').desc(),
               col('totalProfitMargin').desc()
           ] 
        )
    )
).filter(
    col('Rank') <= 3
).show()


+--------------------+--------------------+--------------------+-----------------+----+
|              Region|             Country|        totalRevenue|totalProfitMargin|Rank|
+--------------------+--------------------+--------------------+-----------------+----+
|                Asia|              Bhutan|4.1906714101000005E8|75.61299055704819|   1|
|                Asia|            Mongolia|      3.6116925598E8|67.67414230840438|   2|
|                Asia|           Singapore|3.5950970765999997E8| 73.4750026546758|   3|
|Australia and Oce...|               Tonga|3.4420216186999995E8|67.58031595391664|   1|
|Australia and Oce...|                Fiji|3.4196462662999994E8|72.02509018983733|   2|
|Australia and Oce...|         New Zealand|      3.4134434464E8|78.38372755054573|   3|
|Central America a...|             Grenada|3.2272678528999996E8|64.79109626833556|   1|
|Central America a...|                Cuba|      3.1608106588E8|74.58269993889917|   2|
|Central America a...|          